In [1]:
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer, LoggingHandler, util, evaluation, models, InputExample
from sentence_transformers.datasets import ParallelSentencesDataset
import logging
import os
import csv
import random
import numpy as np
import torch
import pandas as pd
import timeit
from torch.utils.data import DataLoader
from sentence_transformers import SentencesDataset, losses, evaluation, readers

In [2]:
word_embedding_model = models.Transformer('sentence-transformers/stsb-mpnet-base-v2', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Downloading:   0%|          | 0.00/588 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [3]:
trans_df = pd.read_csv('Tagalog/10k/en_tl_10k.csv')

In [8]:
trans_df

TL  \
0     "Hindi ko ugali ang mamulitika; mas gusto kong...   
1     Ayon naman kay Bayan Muna Rep. Carlos Zarate, ...   
2     Ayon naman sa Philippine Embassy sa Washington...   
3     Paano maipagtatanggol ng mga ordinaryong mamam...   
4                                         SM by the bay   
...                                                 ...   
9995  CAMP OLIVAS, Pampanga - Sinibak sa puwesto ang...   
9996  Sa kasalukuyan ay may 21 miyembro ang APEC at ...   
9997  "Bahagi ito ng paninira sa administrasyon at k...   
9998  Ang kampanya sa pangangalap ng lagda ay nagsim...   
9999  Nauna nang dinala sa Bongao upang ibestigahan ...   

                                                    EN   
0     "I do not have the politics; I prefer to be qu...  
1     According to Bayan Muna Rep. Carlos Zarate, Ch...  
2     According to the Philippine Embassy in Washing...  
3     How do ordinary citizens defend themselves fro...  
4                                        SM by the bay   
...                                                 ...  
9995  Camp Olivas, Pampanga - The chief of the court...  
9996  Currently there are 21 members APEC and this is:   
9997  "It is part of the administration of administr...  
9998  The signature recruitment campaign began on Sa...  
9999  Ahead of Bongao to underestimate the two Filip...  

[10000 rows x 2 columns]

In [7]:
trans_df.drop(columns=trans_df.columns[0], 
      axis=1, 
      inplace=True)

In [5]:
sts_df = pd.read_csv('Tagalog/10k/STS_10k.csv')

In [6]:
sts_df

s1  \
0     MANILA, Philippines - Pinamamadali ni Valenzue...   
1     ASINTA ngayon ng Hapee ang solo lead sa 2014-1...   
2     Titiketan pa rin ang mga motoristang lalabag s...   
3     "Sa huli, bawat Pilipino abonado na ni minsan ...   
4     Sinabi ni Sen. Oreta, pinupuri niya ang sibily...   
...                                                 ...   
9995  Mabilis namang tumakas ang suspek na si Reniel...   
9996  Gayunman, sinabi ni Hao na nabuwag ang CHDF na...   
9997  Ayon kay Roberto Kanapi, general manager for e...   
9998  "Katulad nu'ng pinadala sa akin kagabi ng sula...   
9999  Batay sa ipinadalang sulat sa Quezon City Hall...   

                                                     s2  label  
0     Sa hearing ng House Committee on Transportatio...      0  
1     "They could have won their first two games," s...      0  
2     Nakipagpulong na umano si Orbos sa mga traffic...      0  
3     MANILA, Philippines - Kontra ang Malacanang sa...      1  
4     "These officials provided shortcuts in the pro...      1  
...                                                 ...    ...  
9995  Nabatid kay PMSG Dennis Suba, ng Manila Police...      0  
9996  Inihayag pa ni Hao sa kasalukuyan ay wala ng h...      0  
9997  Ayon kay Duterte, isang outrageous attack ang ...      1  
9998  Batay sa naturang order, kailangan munang humi...      1  
9999  "The same request and reason for retention of ...      1  

[10000 rows x 3 columns]

In [9]:
sts_data = []

for index, row in sts_df.iterrows():
  #print(row['label'])
  score = float(row['label']) / 5.0
  sts_data.append(InputExample(texts=[row['s1'], row['s2']], label=score))

In [10]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [11]:
stsb_evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(sts_data, name='sts-benchmark-test', show_progress_bar = True, batch_size = 4, write_csv=True)

logger.info("Original model performance:")
stsb_evaluator(model)

2021-12-07 11:35:48 - Original model performance:
2021-12-07 11:35:48 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-benchmark-test dataset:


Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

2021-12-07 12:02:46 - Cosine-Similarity :	Pearson: -0.3127	Spearman: -0.3248
2021-12-07 12:02:46 - Manhattan-Distance:	Pearson: -0.3223	Spearman: -0.3315
2021-12-07 12:02:46 - Euclidean-Distance:	Pearson: -0.3234	Spearman: -0.3324
2021-12-07 12:02:46 - Dot-Product-Similarity:	Pearson: -0.2962	Spearman: -0.2987


-0.2986585860354109

In [12]:
mse_evaluator = evaluation.MSEEvaluator(trans_df['TL'].values.tolist(),trans_df[trans_df.columns[1]].values.tolist() , teacher_model = model, batch_size=4, show_progress_bar=True, write_csv=True)

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

In [13]:
evaluators = []
evaluators.append(stsb_evaluator)
evaluators.append(mse_evaluator)

In [14]:
train_reader = ParallelSentencesDataset(student_model=model, teacher_model=model)
train_reader.add_dataset(trans_df.values.tolist())

In [15]:
train_dataloader = DataLoader(train_reader, shuffle=True, batch_size=4)
train_loss = losses.MSELoss(model=model)

In [16]:
import datetime

output_path = "output/model-" + datetime.date.today().strftime("%Y-%m-%d")
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluation.SequentialEvaluator(evaluators, main_score_function=lambda scores: scores[-1]),
          epochs=5,
          evaluation_steps=1000,
          warmup_steps=1000,
          scheduler='warmupconstant',
          output_path=output_path,
          save_best_model=True,
          optimizer_params= {'lr': 2e-5, 'eps': 1e-6, 'correct_bias': False}
          )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1367 [00:00<?, ?it/s]

2021-12-07 13:08:31 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-benchmark-test dataset in epoch 0 after 1000 steps:


Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

2021-12-07 21:39:33 - Cosine-Similarity :	Pearson: -0.1151	Spearman: -0.1999
2021-12-07 21:39:33 - Manhattan-Distance:	Pearson: -0.1843	Spearman: -0.2101
2021-12-07 21:39:33 - Euclidean-Distance:	Pearson: -0.1785	Spearman: -0.2038
2021-12-07 21:39:33 - Dot-Product-Similarity:	Pearson: -0.0934	Spearman: -0.1388


Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

2021-12-07 21:46:58 - MSE evaluation (lower = better) on  dataset after epoch 4:
2021-12-07 21:46:58 - MSE (*100):	0.925719


# MODEL DISTILLATION to 128 Dimensions

In [17]:
new_dimension = 128

In [20]:
new_sts_data = []

for index, row in sts_df.iterrows():
  #print(row['label'])
  score = float(row['label']) / 5.0
  new_sts_data.append(row['s1'])
  new_sts_data.append(row['s2']) 

In [21]:
train_embeddings = model.encode(new_sts_data, convert_to_numpy=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [22]:
pca = PCA(n_components=new_dimension)
pca.fit(train_embeddings)
pca_comp = np.asarray(pca.components_)

In [23]:
dense = models.Dense(in_features=model.get_sentence_embedding_dimension(), out_features=new_dimension, bias=False, activation_function=torch.nn.Identity())
dense.linear.weight = torch.nn.Parameter(torch.tensor(pca_comp))
model.add_module('dense', dense)

In [28]:
logger.info("Model with {} dimensions:".format(new_dimension))
stsb_evaluator(model)

2021-12-08 18:13:56 - Model with 128 dimensions:
2021-12-08 18:13:56 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-benchmark-test dataset:


Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

Batches:   0%|          | 0/2500 [00:00<?, ?it/s]

2021-12-08 18:40:14 - Cosine-Similarity :	Pearson: -0.1126	Spearman: -0.2022
2021-12-08 18:40:14 - Manhattan-Distance:	Pearson: -0.1977	Spearman: -0.2217
2021-12-08 18:40:14 - Euclidean-Distance:	Pearson: -0.1825	Spearman: -0.2070
2021-12-08 18:40:14 - Dot-Product-Similarity:	Pearson: -0.0872	Spearman: -0.1298


-0.12978309723352843

In [25]:
model.save('models/my-128dim-model')

2021-12-08 11:30:36 - Save model to models/my-128dim-model
